In [8]:
# 68 face points

from collections import OrderedDict
import numpy as np
import argparse
import dlib
import cv2

### params ###

--shape-predictor shape_predictor_68_face_landmarks.dat
--image images/human_images/img_1.jpeg

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
ap.add_argument("-i", "--image", required=True, help="path to input image")
args = vars(ap.parse_args())


In [10]:
FACIAL_LANDMARKS_68_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

FACIAL_LANDMARKS_5_IDXS = OrderedDict([
    ("right_eye", (2, 3)),
    ("left_eye", (0, 1)),
    ("nose", (4))
])

In [11]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    return coords

In [12]:
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    overlay = image.copy()
    output = image.copy()

    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                  (168, 100, 168), (158, 163, 32),
                  (163, 38, 32), (180, 42, 220)]

    # Iterate over each facial landmark region
    for (i, name) in enumerate(FACIAL_LANDMARKS_68_IDXS.keys()):
        # Get the coordinates of each point in the region
        (j, k) = FACIAL_LANDMARKS_68_IDXS[name]
        pts = shape[j:k]

        # Check the position
        if name == "jaw":
            # Connect points with lines
            for l in range(1, len(pts)):
                ptA = tuple(pts[l - 1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)
        else:
            # Calculate convex hull
            hull = cv2.convexHull(pts)
            cv2.drawContours(overlay, [hull], -1, colors[i], -1)

    # Overlay on the original image with specified ratio
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    return output

In [13]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

image = cv2.imread('../images/human_images/img_1.jpeg')
(h, w) = image.shape[:2]
width = 500
r = width / float(w)
dim = (width, int(h * r))
image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)